## 1. Importing important libraries for LIME

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import torch.nn as nn
import numpy as np
import os,json

import torch
from torchvision import models, transforms
from torch.autograd import Variable
import torch.nn.functional as F

from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

## Loading the saved model

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
class TinyVGG(nn.Module):
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
    super().__init__()
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2)
    )
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(2)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = hidden_units*32*32,
                  out_features=output_shape)
    )
  def forward(self, x:torch.Tensor):
    x = self.conv_block_1(x)
    x = self.conv_block_2(x)
    x = self.classifier(x)
    return x

torch.manual_seed(42)
model_BCE_20epochs = TinyVGG(input_shape=3,
                             hidden_units=10,
                             output_shape=1).to(device)
model_BCE_20epochs

In [ ]:
#BCE_CNNmodel = torch.load(f="path_of_your_saved_CNN_model.pth", map_location=torch.device("cpu"))
#BCE_CNNmodel

In [ ]:
data_transform = transforms.Compose([
    transforms.Resize(size=(128,128), antialias=None), #resize image
    transforms.RandomHorizontalFlip(p=0.5), #flip the images randomly horizontally
    transforms.ToTensor()
])

In [ ]:
# Loading and transforming data using datasets.ImageFolder
train_data_cl = datasets.ImageFolder(root='add_your_training_path',
                                     transform=data_transform, #a transform for the data
                                     target_transform=None) #a transform for the label/target
val_data_cl = datasets.ImageFolder(root='add_your_validation_path',
                                   transform=data_transform)
train_data_cl, val_data_cl

In [ ]:
import requests

# Download helper functions from Learn PyTorch repo (if not already downloaded)
if os.path.isfile("helper_function_script_at_given_path.py"):
  print("helper_functions.py already exists, skipping download")
else:
  print("Downloading helper_functions.py")
  os.chdir("given_path_for_helper_function_script")
  # Note: you need the "raw" GitHub URL for this to work
  request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
  with open("helper_functions.py", "wb") as f:
    f.write(request.content)

In [ ]:
# Create a new instance of TinyVGG (the same class as our saved state_dict())
# Note: loading model will error if the shapes here aren't the same as the saved version
loaded_model_for_LIME = TinyVGG(input_shape=3,
                                hidden_units=10, # try changing this to 128 and seeing what happens
                                output_shape=1).to(device)
# Load in the saved state_dict()
loaded_model_for_LIME.load_state_dict(torch.load(f="path_of_your_saved_CNN_model.pth", map_location=torch.device("cpu")))

# send model to target device
loaded_model_for_LIME.to(device)

## Load a sample image and view it

In [ ]:
import os
import random

def select_random_image(folder_path):
    # Get a list of all files in the folder
    all_files = os.listdir(folder_path)

    # Filter the list to include only image files (you may need to adjust this based on your file types)
    image_files = [file for file in all_files if file.lower().endswith(('.png', '.jpg', '.jpeg'))]

    if not image_files:
        print("No image files found in the specified folder.")
        return None

    # Randomly select an image from the list
    selected_image = random.choice(image_files) #image_files[3]#

    # Construct the full path to the selected image
    image_path = os.path.join(folder_path, selected_image)

    return image_path, selected_image

def plot_image(image_path,destination,selected_image):
    # Load and plot the image
    img = plt.imread(image_path)
    imgplot = plt.imshow(img)
    plt.axis('off')  # Turn off axis labels
    plt.savefig( os.path.join(destination, selected_image)+"_OriginalPic.png", bbox_inches='tight')
    plt.show()

# Example usage:
folder_path = "path_of_test_dataset_specific_class"
random_image, selected_filename = select_random_image(folder_path)
destination_path = "path_to_save_LIME_output"
if random_image:
    print(f"Randomly selected image: {random_image}")
    plot_image(random_image,destination_path,selected_filename)
else:
    print("No image selected.")

In [ ]:
def get_image(path):
  with open(os.path.abspath(path), 'rb') as f:
    with Image.open(f) as img:
      return img.convert('RGB')
# taking an image from the test set
img = get_image(random_image)
plt.imshow(img)
plt.axis("off")

In [ ]:
# we will convert the above image to tensor and apply transformations similar to loaded model
def get_input_transform(target_size = (128,128)):
  return transforms.Compose([
      transforms.Resize(target_size),
      transforms.ToTensor(),
  ])
  return transf
def get_input_tensors(img):
  transf = get_input_transform()
  #unsqueeze converts single image to batch of 1
  return transf(img).unsqueeze(0)

### Load labels for prediction

In [ ]:
class_info = {0:'crop', 1:'weed'}
class_info[1]

### Get the prediction for the image

In [ ]:
img_t = get_input_tensors(img).to(device)
loaded_model_for_LIME.eval()
logits = loaded_model_for_LIME(img_t)

In [ ]:
# Apply sigmoid activation function element-wise
probs = torch.sigmoid(logits)
probs

### LIME:
We are getting ready to use Lime. Lime produces the array of images from original input image by pertubation algorithm. So we need to provide two things: (1) original image as numpy array (2) classification function that would take array of purturbed images as input and produce the probabilities for each class for each image as output.

For Pytorch, first we need to define two separate transforms: (1) to take PIL image, resize and crop it (2) take resized, cropped image and apply whitening.

In [ ]:
def get_pil_transform():
    transf = transforms.Compose([
        transforms.Resize((128, 128)),
    ])
    return transf

def get_preprocess_transform():
    transf = transforms.Compose([
        transforms.ToTensor(),
    ])
    return transf

pill_transf = get_pil_transform()
preprocess_transform = get_preprocess_transform()

In [ ]:
pill_transf, preprocess_transform

Now we are ready to define classification function that Lime needs. The input to this function is numpy array of images where each image is ndarray of shape (channel, height, width). The output is numpy aaray of shape (image index, classes) where each value in array should be probability for that image, class combination.

classifier prediction probability function, which takes a numpy array and outputs prediction probabilities. (LIME documentation)

https://lime-ml.readthedocs.io/en/latest/lime.html

In [ ]:
def batch_predict(images):
  loaded_model_for_LIME.eval()
  batch = torch.stack(tuple(preprocess_transform(i) for i in images)) #dim=0)

  #device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  loaded_model_for_LIME.to(device)
  batch = batch.to(device)

  logits = loaded_model_for_LIME(batch)
  probs_predict = torch.sigmoid(logits) #F.torch.sigmoid(logits)
  return probs_predict.detach().cpu().numpy()

In [ ]:
# let's test our function on the sample image
test_pred = batch_predict([pill_transf(img)])

Import LIME and create explanations

In [ ]:
pip install lime

In [ ]:
from lime import lime_image

In [ ]:
np.array(pill_transf(img)).shape

In [ ]:
img_transformed = pill_transf(img)
print(type(img_transformed)) #he output of pill_transf(img) is a PIL Image object.
#In order to use LIME with this image data, you need to convert it to a NumPy array.

In [ ]:
img_pil = pill_transf(img)

# Convert the PIL image to NumPy array
img_array = np.array(img_pil)

# Printing the datatype of img_array
print(type(img_array))

**explain_instance:** Generates explanations for a prediction.

First, we generate neighborhood data by randomly perturbing features from the instance (see __data_inverse). We then learn locally weighted linear models on this neighborhood data to explain each of the classes in an interpretable way (see lime_base.py). (LIME documentation)

In [ ]:
explainer = lime_image.LimeImageExplainer() #creating an explainer object
explanation = explainer.explain_instance(img_array,#np.array(pill_transf(img)), # image instance you want to explain
                                         batch_predict, #classification function
                                         top_labels=1,
                                         hide_color=0,
                                         num_samples=1000 #number of images that will be sent to the classification function
                                         )
#neighborhood_labels = np.array(explanation.local_pred.ravel())

In [ ]:
from skimage.segmentation import mark_boundaries

In [ ]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=True, num_features=10, hide_rest=False)
fig, ax = plt.subplots(1,2, figsize=(9,9))
ax[0].imshow(mark_boundaries(temp /255,np.flipud(mask)))
ax[0].axis('off')
ax[1].imshow(img_transformed) #this is pill transformed vs -- what is img_t??
ax[1].axis('off')
plt.savefig(os.path.join(destination_path, selected_filename)+"_Explanation.png", bbox_inches="tight")

In [ ]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, min_weight=0.0, num_features=10, hide_rest=True, negative_only=False)
img_boundary2 = mark_boundaries(temp/255, np.flipud(mask))
fig, ax = plt.subplots(1,2, figsize=(9,9))
ax[0].imshow(img_boundary2)
#plt.imshow(img_boundary2)
ax[0].axis("off")
ax[1].imshow(img_transformed)
ax[1].axis("off")
plt.savefig(os.path.join(destination_path, selected_filename)+"_Hide_rest_trueExplanation.png", bbox_inches="tight")
# output doesn't show red color
#i.e. area which is not contributing towards the prediction

In [ ]:
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, min_weight=0.0, num_features=10, hide_rest=False, negative_only=True)
img_boundary2 = mark_boundaries(temp/255, np.flipud(mask))
fig, ax = plt.subplots(1,2, figsize=(9,9))
ax[0].imshow(img_boundary2)
#plt.imshow(img_boundary2)
ax[0].axis("off")
ax[1].imshow(img_transformed)
ax[1].axis("off")
plt.savefig(os.path.join(destination_path, selected_filename)+"_Hide_rest_false+ negative_only_true Explanation.png", bbox_inches="tight")

In [ ]:
temp, mask = explanation.get_image_and_mask(0, positive_only=True, num_features=5, hide_rest=False)
plt.imshow(mark_boundaries(temp / 255 + 0.2, mask))